# Chapter 3 Exercises

Import of book's data structures to be used. Renamed the new list type to NewList becuse scala is weird.

In [1]:
sealed trait NewList[+A] // `NewList` data type, parameterized on a type, `A`
case object Nil extends NewList[Nothing] // A `NewList` data constructor representing the empty list

case class Cons[+A](head: A, tail: NewList[A]) extends NewList[A]

object NewList { // `NewList` companion object. Contains functions for creating and working with lists.
    def sum(ints: NewList[Int]): Int = ints match { // A function that uses pattern matching to add up a list of integers
        case Nil => 0 // The sum of the empty list is 0.
        case Cons(x,xs) => x + sum(xs) // The sum of a list starting with `x` is `x` plus the sum of the rest of the list.
    }

    def product(ds: NewList[Double]): Double = ds match {
        case Nil => 1.0
        case Cons(0.0, _) => 0.0
        case Cons(x,xs) => x * product(xs)
    }

    def apply[A](as: A*): NewList[A] = // Variadic function syntax
        if (as.isEmpty) Nil
        else Cons(as.head, apply(as.tail: _*))

    def append[A](a1: NewList[A], a2: NewList[A]): NewList[A] =
        a1 match {
            case Nil => a2
            case Cons(h,t) => Cons(h, append(t, a2))
        }

    def foldRight[A,B](as: NewList[A], z: B)(f: (A, B) => B): B = // Utility functions
        as match {
            case Nil => z
            case Cons(x, xs) => f(x, foldRight(xs, z)(f))
        }

    def sum2(ns: NewList[Int]) =
        foldRight(ns, 0)((x,y) => x + y)

    def product2(ns: NewList[Double]) =
        foldRight(ns, 1.0)(_ * _) // `_ * _` is more concise notation for `(x,y) => x * y`; see sidebar

    // Exercise 3.2
    def tail[A](x: NewList[A]): NewList[A] = {
        x match {
            case Nil => sys.error("Empty NewList, no tail exists in an empty NewList")
            case Cons(_, t) => t
        }
    }
    
    // Exercise 3.3
    def setHead[A](l: NewList[A], h: A): NewList[A] = {
        l match {
            case Nil => sys.error("Empty NewList, no first element to replace")
            case Cons(_, t) => Cons(h, t)
        }
    }

    // Exercise 3.4
    def drop[A](l: NewList[A], n: Int): NewList[A] = {
        if(n <= 0) {
            l
        } else {
            l match {
                case Nil => sys.error("Empty NewList, no elements to drop")
                case Cons(_, t) => NewList.drop(t, n - 1)
            }
        }
    }

    // Exercise 3.5
    def dropWhile[A](l: NewList[A], f: A => Boolean): NewList[A] = {
        l match {
            case Nil => sys.error("Empty NewList, no elements to drop")
            case Cons(h, t) if(f(h)) => { 
                NewList.dropWhile(t, f)
            }
            case _ => l
        }
    }

    // Exercise 3.6
    def init[A](l: NewList[A]): NewList[A] = {
        l match {
            case Nil => sys.error("Empty NewList, no elements to drop")
            case Cons(_, Nil) => Nil
            case Cons(h, t) => Cons(h, init(t))
        }
    }

    // Exercise 3.9
    def length[A](l: NewList[A]): Int = {
        NewList.foldRight(l, 0)((_, count) => count + 1)
    }

    // Exercise 3.10
    @annotation.tailrec
    def foldLeft[A,B](l: NewList[A], z: B)(f: (B, A) => B): B = {
        l match {
            case Nil => z
            case Cons(h, t) => NewList.foldLeft(t, f(z, h))(f)
        }
    }
    
    // Exercise 3.11
    def sum3(ns: NewList[Int]) = {
        foldLeft(ns, 0)((x,y) => x + y)
    }
    def product3(ns: NewList[Double]) = {
        foldLeft(ns, 1.0)((x,y) => x * y)
    }
    def length2[A](l: NewList[A]): Int = {
        foldLeft(l, 0)((count, _) => count + 1)
    }
    
    // Exercise 3.12
    def reverseFirstTry[A](l: NewList[A]): NewList[A] = {
        l match {
            case Nil => Nil
            case Cons(h, t) => append(reverse(t), NewList(h))
        }
    }
    
    // Exercise 3.12 continued
    def reverse[A](l: NewList[A]): NewList[A] = {
        foldLeft(l, NewList[A]())((l, h) => Cons(h,l))
    }
    
    // Exercise 3.13
    def foldRightViaFoldLeft[A,B](l: NewList[A], z: B)(f: (A,B) => B): B = {
        foldLeft(reverse(l), z)((b,a) => f(a,b))
    }
    
    // Exercise 3.14
    def appendViaFoldRight[A](l: NewList[A], r: NewList[A]): NewList[A] = {
        foldRight(l, r)(Cons(_,_))
    }
    
    // Exercise 3.15
    def concatenate[A](l: NewList[NewList[A]]): NewList[A] = {
        foldRight(l, Nil:NewList[A])(append)
    }
    
    // Exercise 3.16
    def addOneToList(l: NewList[Int]): NewList[Int] = {
        foldRight(l, Nil:NewList[Int])((h, t) => Cons(h + 1, t))
    }
    
    // Exercise 3.17
    def convertDoublesToStrings(l: NewList[Double]): NewList[String] = {
        foldRight(l, Nil:NewList[String])((h, t) => Cons(h.toString, t))
    }
    
    // Exercise 3.18
    def map[A,B](l: NewList[A])(f: A => B): NewList[B] = {
        foldRight(l, Nil:NewList[B])((h, t) => Cons(f(h), t))
    }
    
    // Exercise 3.19
    def filter[A](as: NewList[A])(f: A => Boolean): NewList[A] = {
        foldRight(as, Nil:NewList[A])((h, t) => if(f(h)) {
            Cons(h, t)
        } else {
            t
        })
    }
    
    // Exercise 3.20
    def flatMap[A, B](as: NewList[A])(f: A => NewList[B]): NewList[B] = {
        concatenate(map(as)(f))
    }
    
    // Exercise 3.21
    def filterByFlatMap[A](l: NewList[A])(f: A => Boolean): NewList[A] = {
        flatMap(l)(x => if(f(x)) {
            NewList(x)
        } else {
            Nil
        })
    }
    
    // Exercise 3.22
    def addNewLists(a: NewList[Int], b: NewList[Int]): NewList[Int] = {
        (a,b) match {
            case (Nil, _) => Nil
            case (_, Nil) => Nil
            case (Cons(a1, b1), Cons(a2, b2)) => Cons(a1 + a2, addNewLists(b1, b2))
        }
    }
    
    // Exercise 3.23
    def zipWith[A,B,C](a: NewList[A], b: NewList[B])(f: (A,B) => C): NewList[C] = {
        (a,b) match {
            case (Nil, _) => Nil
            case (_, Nil) => Nil
            case (Cons(a1, b1), Cons(a2, b2)) => Cons(f(a1, a2), zipWith(b1, b2)(f))
        }
    }
    
    // Exercise 3.24
    @annotation.tailrec
    def startsWith[A](l: NewList[A], prefix: NewList[A]): Boolean = {
        (l, prefix) match {
            case (_, Nil) => true
            case (Cons(h, t), Cons(h2, t2)) if(h == h2) => startsWith(t, t2)
            case _ => false
        }
    }
    
    @annotation.tailrec
    def hasSubsequence[A](long: NewList[A], sub: NewList[A]): Boolean = {
        long match {
            case Nil => sub == Nil
            case _ if startsWith(long, sub) => true
            case Cons(_, t) => hasSubsequence(t, sub)
        }
    }
}


defined trait NewList
defined object Nil
defined class Cons
defined object NewList

#### Exercise 3.1

In [2]:
val x = NewList(1,2,3,4,5) match {
    case Cons(x, Cons(2, Cons(4, _))) => x
    case Nil => 42
    case Cons(x, Cons(y, Cons(3, Cons(4, _)))) => x + y
    case Cons(h, t) => h + NewList.sum(t)
    case _ => 101
}

x: Int = 3

This returns 3 because it hits the third case where we have (x,y,3,4,_) and returns x + y (1 + 2) here. The other cases don't match a NewList of 1 to 5

#### Exercise 3.2

In [3]:
val a = NewList(1,2,3)
NewList.tail(a)

a: NewList[Int] = Cons(1,Cons(2,Cons(3,Nil)))
res2_1: NewList[Int] = Cons(2,Cons(3,Nil))

In [4]:
val a = NewList(1)
NewList.tail(a)

a: NewList[Int] = Cons(1,Nil)
res3_1: NewList[Int] = Nil

In [5]:
val a = NewList()
NewList.tail(a)

: 

#### Exercise 3.3

In [6]:
val a = NewList(1,2,3)
NewList.setHead(a, 10)

a: NewList[Int] = Cons(1,Cons(2,Cons(3,Nil)))
res5_1: NewList[Int] = Cons(10,Cons(2,Cons(3,Nil)))

In [7]:
val a = NewList(1)
NewList.setHead(a, 10)

a: NewList[Int] = Cons(1,Nil)
res6_1: NewList[Int] = Cons(10,Nil)

In [8]:
val a = NewList()
NewList.setHead(a, 10)

: 

#### Exercise 3.4

In [9]:
val a = NewList(1,2,3)
NewList.drop(a, 1)
NewList.drop(a, 0)

a: NewList[Int] = Cons(1,Cons(2,Cons(3,Nil)))
res8_1: NewList[Int] = Cons(2,Cons(3,Nil))
res8_2: NewList[Int] = Cons(1,Cons(2,Cons(3,Nil)))

In [10]:
NewList.drop(NewList(), 1)

: 

In [11]:
NewList.drop(NewList(1,2,3,4,5,6,7), 6)

res10: NewList[Int] = Cons(7,Nil)

#### Exercise 3.5

In [12]:
// Assume we're always passing it an integer 
def logicalFunction(a: Int): Boolean = {
    if(a > 5) { 
        true
    } else {
        false
    }
}

defined function logicalFunction

In [13]:
NewList.dropWhile(NewList(8,7,6,5,4,3,2,1), logicalFunction)

res12: NewList[Int] = Cons(5,Cons(4,Cons(3,Cons(2,Cons(1,Nil)))))

In [14]:
NewList.dropWhile(NewList(), logicalFunction)

: 

In [15]:
NewList.dropWhile(NewList(1,2,3,4), logicalFunction)

res14: NewList[Int] = Cons(1,Cons(2,Cons(3,Cons(4,Nil))))

In [16]:
NewList.dropWhile(NewList(1,2,3,4,5,6,7,8,9), logicalFunction)

res15: NewList[Int] = Cons(1,Cons(2,Cons(3,Cons(4,Cons(5,Cons(6,Cons(7,Cons(8,Cons(9,Nil)))))))))

#### Exercise 3.6

In [17]:
NewList.init(NewList(1,2,3,4))
NewList.init(NewList(1))

res16_0: NewList[Int] = Cons(1,Cons(2,Cons(3,Nil)))
res16_1: NewList[Int] = Nil

In [18]:
NewList.init(NewList())

: 

#### Exercise 3.7

No, this does not seem possible. The answer key confirmed that by saying in fold right we evaluate the arguments first before ever calling f, so we traverse the entire list before even calling f for the "first" time. 

#### Exercise 3.8

In [19]:
NewList.foldRight(NewList(1,2,3), Nil:NewList[Int])(Cons(_,_))

res18: NewList[Int] = Cons(1,Cons(2,Cons(3,Nil)))

This folds right a NewList of integers onto a Nil NewList of integers with the Cons(\_,\_) which makes it behave basically as an identity function

#### Exercise 3.9

In [20]:
NewList.length(NewList(1,2,3,4,5))
NewList.length(NewList())

res19_0: Int = 5
res19_1: Int = 0

#### Exercise 3.10

In [21]:
NewList.foldLeft(NewList(1,2), 10)((a, b) => a + b)
NewList.foldLeft(NewList(1,2,2), 22)((a, b) => a + b)

res20_0: Int = 13
res20_1: Int = 27

#### Exercise 3.11

In [22]:
NewList.sum3(NewList(1,2,3))
NewList.product3(NewList(1,2,3))
NewList.length2(NewList(1,2,3))

res21_0: Int = 6
res21_1: Double = 6.0
res21_2: Int = 3

#### Exercise 3.12

In [23]:
NewList.reverseFirstTry(NewList(1,2,3,4))

res22: NewList[Int] = Cons(4,Cons(3,Cons(2,Cons(1,Nil))))

My first (dirty) try of reverse works, which is cool :) 

And using foldLeft with a little help...

In [24]:
NewList.reverse(NewList(1,2,3,4))

res23: NewList[Int] = Cons(4,Cons(3,Cons(2,Cons(1,Nil))))

#### Exercise 3.13

With some help from the answer key, here is one working implementation of foldRightViaFoldLeft and the reverse function written in the previous exercise

In [25]:
NewList.foldRightViaFoldLeft(NewList(1,2,3), Nil:NewList[Int])(Cons(_,_))

res24: NewList[Int] = Cons(1,Cons(2,Cons(3,Nil)))

#### Exercise 3.14

In [26]:
NewList.appendViaFoldRight(NewList(2,3,4), NewList(6,5))

res25: NewList[Int] = Cons(2,Cons(3,Cons(4,Cons(6,Cons(5,Nil)))))

#### Exercise 3.15

In [27]:
val listOfNewLists = NewList(NewList(1,2), NewList(3,4), NewList(5,6))

listOfNewLists: NewList[NewList[Int]] = Cons(Cons(1,Cons(2,Nil)),Cons(Cons(3,Cons(4,Nil)),Cons(Cons(5,Cons(6,Nil)),Nil)))

In [28]:
NewList.concatenate(listOfNewLists)

res27: NewList[Int] = Cons(1,Cons(2,Cons(3,Cons(4,Cons(5,Cons(6,Nil))))))

#### Exercise 3.16

In [29]:
NewList.addOneToList(NewList(1,2,3,4,5))
NewList.addOneToList(NewList())

res28_0: NewList[Int] = Cons(2,Cons(3,Cons(4,Cons(5,Cons(6,Nil)))))
res28_1: NewList[Int] = Nil

#### Exercise 3.17

In [30]:
NewList.convertDoublesToStrings(NewList(1.0,2.0,3.0))

res29: NewList[String] = Cons(1.0,Cons(2.0,Cons(3.0,Nil)))

#### Exercise 3.18

In [31]:
// Using the logicalFunction defined above, demonstrate map
NewList.map(NewList(1,2,3,4,5,6,7))(logicalFunction)

res30: NewList[Boolean] = Cons(false,Cons(false,Cons(false,Cons(false,Cons(false,Cons(true,Cons(true,Nil)))))))

#### Exercise 3.19

In [32]:
def isEven(a: Int): Boolean = {
    if(a % 2 == 0) { 
        true
    } else {
        false
    }
}

defined function isEven

In [33]:
NewList.filter(NewList(1,2,3,4,5,6,7,8))(isEven)

res32: NewList[Int] = Cons(2,Cons(4,Cons(6,Cons(8,Nil))))

#### Exercise 3.20

In [34]:
NewList.flatMap(NewList(1,2,3))(i => NewList(i, i))

res33: NewList[Int] = Cons(1,Cons(1,Cons(2,Cons(2,Cons(3,Cons(3,Nil))))))

#### Exercise 3.21

In [35]:
NewList.filterByFlatMap(NewList(1,2,3,4,5,6,7,8))(isEven)

res34: NewList[Int] = Cons(2,Cons(4,Cons(6,Cons(8,Nil))))

#### Exercise 3.22

In [36]:
NewList.addNewLists(NewList(1,2,3), NewList(1,2,3))
NewList.addNewLists(NewList(), NewList(1,2,3))
NewList.addNewLists(NewList(1,2,3), NewList())

res35_0: NewList[Int] = Cons(2,Cons(4,Cons(6,Nil)))
res35_1: NewList[Int] = Nil
res35_2: NewList[Int] = Nil

#### Exercise 3.23

In [37]:
NewList.zipWith(NewList(1,2,3), NewList(1,2,3))((a, b) => a + b)

res36: NewList[Int] = Cons(2,Cons(4,Cons(6,Nil)))

#### Exercise 3.24

Implementation with some help from the answers

In [38]:
NewList.hasSubsequence(NewList(1,2,3,4,5,5,6,7), NewList(5,5))
NewList.hasSubsequence(NewList(1,2,3,4,5,5,6,7), NewList(5,5,2))

res37_0: Boolean = true
res37_1: Boolean = false

## Trees

In [39]:
sealed trait Tree[+A]
case class Leaf[A](value: A) extends Tree[A]
case class Branch[A](left: Tree[A], right: Tree[A]) extends Tree[A]

// Defined with liberal help from the solutions but studied deeply

object Tree {

    // Exercise 3.25
    def size[A](t: Tree[A]): Int = { 
        t match {
            case Leaf(_) => 1
            case Branch(l,r) => 1 + size(l) + size(r)
        }
    }

    // Exercise 3.26
    def maximum(t: Tree[Int]): Int = {
        t match {
            case Leaf(n) => n
            case Branch(l,r) => maximum(l) max maximum(r)
        }
    }

    // Exercise 3.27
    def depth[A](t: Tree[A]): Int = {
        t match {
            case Leaf(_) => 0
            case Branch(l,r) => 1 + (depth(l) max depth(r))
        }
    }

    // Exercise 3.28
    def map[A,B](t: Tree[A])(f: A => B): Tree[B] = {
        t match {
            case Leaf(a) => Leaf(f(a))
            case Branch(l,r) => Branch(map(l)(f), map(r)(f))
        }
    }

    // Exercise 3.29
    def fold[A,B](t: Tree[A])(f: A => B)(g: (B,B) => B): B = {
        t match {
            case Leaf(a) => f(a)
            case Branch(l,r) => g(fold(l)(f)(g), fold(r)(f)(g))
        }
    }

    def sizeViaFold[A](t: Tree[A]): Int = {
        fold(t)(a => 1)(1 + _ + _)
    }

    def maximumViaFold(t: Tree[Int]): Int = {
        fold(t)(a => a)(_ max _)
    }

    def depthViaFold[A](t: Tree[A]): Int = {
        fold(t)(a => 0)((d1,d2) => 1 + (d1 max d2))
    }

    def mapViaFold[A,B](t: Tree[A])(f: A => B): Tree[B] = {
        fold(t)(a => Leaf(f(a)): Tree[B])(Branch(_,_))
    }
}

defined trait Tree
defined class Leaf
defined class Branch
defined object Tree

#### Exercise 3.25

In [40]:
val T = Branch(Branch(Leaf(200),
                      Leaf(5)), 
               Branch(Leaf(10), 
                      Branch(Leaf(25),
                             Leaf(2))))
Tree.size(T)

T: Branch[Int] = Branch(Branch(Leaf(200),Leaf(5)),Branch(Leaf(10),Branch(Leaf(25),Leaf(2))))
res39_1: Int = 9

In [41]:
val T2 = Branch(Leaf(2000), 
                Branch(Leaf(10), 
                       Leaf(25)))
Tree.size(T2)

T2: Branch[Int] = Branch(Leaf(2000),Branch(Leaf(10),Leaf(25)))
res40_1: Int = 5

#### Exercise 3.26

In [42]:
Tree.maximum(T)
Tree.maximum(T2)

res41_0: Int = 200
res41_1: Int = 2000

#### Exercise 3.27

In [43]:
Tree.depth(T)
Tree.depth(T2)

res42_0: Int = 3
res42_1: Int = 2

#### Exercise 3.28

In [44]:
Tree.map(T)(logicalFunction)
Tree.map(T2)(logicalFunction)

res43_0: Tree[Boolean] = Branch(Branch(Leaf(true),Leaf(false)),Branch(Leaf(true),Branch(Leaf(true),Leaf(false))))
res43_1: Tree[Boolean] = Branch(Leaf(true),Branch(Leaf(true),Leaf(true)))

#### Exercise 3.29

In [45]:
Tree.sizeViaFold(T)
Tree.sizeViaFold(T2)

res44_0: Int = 9
res44_1: Int = 5

In [46]:
Tree.maximumViaFold(T)
Tree.maximumViaFold(T2)

res45_0: Int = 200
res45_1: Int = 2000

In [47]:
Tree.depthViaFold(T)
Tree.depthViaFold(T2)

res46_0: Int = 3
res46_1: Int = 2

In [48]:
Tree.mapViaFold(T)(logicalFunction)
Tree.mapViaFold(T2)(logicalFunction)

res47_0: Tree[Boolean] = Branch(Branch(Leaf(true),Leaf(false)),Branch(Leaf(true),Branch(Leaf(true),Leaf(false))))
res47_1: Tree[Boolean] = Branch(Leaf(true),Branch(Leaf(true),Leaf(true)))